<a href="https://colab.research.google.com/github/rahulbordoloi/BERT-Transformer/blob/master/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis using BERT

* Author: Rahul Bordoloi 
* Email-ID : <rahulbordoloi24@gmail.com>, <mail@rahulbordoloi.me>                                                                             
* Date Created: 30 August, 2020      
* Language & Version - Python 3.8.4    

## Project Outline

1. Setting up Directory

2. Importing Libraries and Preprocessing

3. Training/Validation Split

4. Loading Tokenizer and Encoding our Data

5.: Setting up BERT Pretrained Model

6. Creating Data Loaders

7. Setting Up Optimizer and Scheduler

8. Defining our Performance Metrics

9. Creating our Training Loop

10. Loading and Evaluating our Model

*  We will use the SMILE Twitter dataset.

_Wang, Bo; Tsakalidis, Adam; Liakata, Maria; Zubiaga, Arkaitz; Procter, Rob; Jensen, Eric (2016): SMILE Twitter Emotion dataset. figshare. <br>
Dataset -  https://doi.org/10.6084/m9.figshare.3187909.v2_

## About **BERT**

BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks.

For more information, Follow the Links - 

*   [Original Paper](https://arxiv.org/abs/1810.04805) 
*   [HuggingFace Documentation](https://huggingface.co/transformers/model_doc/bert.html)
*   [Bert Documentation](https://characters.fandom.com/wiki/Bert_(Sesame_Street)






BERT Architecture

<a href="https://ibb.co/vdnhQjm"><img src="https://i.ibb.co/fNw9nxt/BERT-diagrams.png" alt="BERT-diagrams" border="0"></a>

# 1. Setting up Directory

In [1]:
# Mounting GDrive to Colab Notebook
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# Present Working Directory
!pwd

/content


In [3]:
# Changing CWD
import os
os.chdir('/content/gdrive/My Drive/Data Science/NLP/BERT')

In [4]:
# Contents of CWD
!ls

Data  Images  Models  Project.ipynb


# 2. Importing Libraries and Preprocessing

In [5]:
# Supress all Warnings
import warnings
warnings.filterwarnings('ignore')

In [6]:
# Installing Transformers to the Kernel
!pip install transformers

     |████████████████████████████████| 778kB 5.3MB/s 
     |████████████████████████████████| 890kB 30.1MB/s 
     |████████████████████████████████| 1.1MB 44.7MB/s 
     |████████████████████████████████| 3.0MB 53.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=3b7c9c2e1d7b9c2a370ff43d8f661882e660c64e7824dcb1859bc10525030580
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [7]:
# Importing Libraries

## Common General Purpose Libraries
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import random
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

## Importing Optimizers and Schedulers
from transformers import AdamW, get_linear_schedule_with_warmup

## Importing dataloaders
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

## Importing BERT Pre-Trained Model
from transformers import BertForSequenceClassification

## Importing Tokenizers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [8]:
# Importing Dataset
df = pd.read_csv('Data/smile-annotations-final.csv', names = ['id', 'text', 'category'], error_bad_lines = False, low_memory = True, verbose = -1)
df.set_index('id', inplace = True)

Tokenization took: 1.74 ms
Type conversion took: 3.29 ms
Parser memory cleanup took: 0.01 ms


In [9]:
# Exploring Data
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [10]:
# Visualising Text of 1st Record
df.text.iloc[0]

'@aandraous @britishmuseum @AndrewsAntonio Merci pour le partage! @openwinemap'

In [11]:
# Counts of Categories Present
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [12]:
# Taking in Account only Single Type Categories for Convenience
df = df[(~df.category.str.contains('\|')) & (df.category != 'nocode')]

In [13]:
# Counts of Categories Present after Removal
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [14]:
# Encoding Categorical Label with Numbers
possible_labels = df.category.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label] = index

In [15]:
# Dictionary for the Values wrt Label
label_dict

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [16]:
# Mapping the Categories with Numerical Values
df['label'] = df.category.replace(label_dict)
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


# 3. Training/Validation Split

In [17]:
# Splitting into Train and Validation sets for Model

x_train, x_val, y_train, y_val = train_test_split(
    df.index.values,            # Uniquely Indentify Each Row
    df.label.values,            # Make Random Split Based on Label and Index
    test_size = 0.15,
    random_state = 17,
    stratify = df.label.values
    )

In [18]:
# To Tell whether that row is used for Training or Validation

df['data_type'] = ['not_set'] * df.shape[0]

# Mapping into Train-Validation
df.loc[x_train, 'data_type'] = 'train'
df.loc[x_val, 'data_type'] = 'val'

In [19]:
# Checking out Counts of Each Categories acorss label and test-val sets
df.groupby(['category', 'label', 'data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

# 4. Loading Tokenizer and Encoding our Data

In [20]:
# Setting up Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True) # Bert-Base-Uncased is a Pre-Trained model of BERT

In [21]:
# Convert our Tweets from Language to Encoded Form

## Train
encoded_data_train = tokenizer.batch_encode_plus(df[df.data_type == 'train'].text.values, add_special_tokens = True,
                                                 return_attention_mask = True, pad_to_max_length = True, max_length = 256,
                                                 return_tensors = 'pt')

## Validation
encoded_data_val = tokenizer.batch_encode_plus(df[df.data_type == 'val'].text.values, add_special_tokens = True,
                                                 return_attention_mask = True, pad_to_max_length = True, max_length = 256,
                                                 return_tensors = 'pt')

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [22]:
# Setting up Inputs for BERT

## Train
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type == 'train'].label.values)

## Validation
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type == 'val'].label.values)

In [23]:
# Datasets for NLP using BERT - Converting into Tensors Form
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [24]:
# Shape
len(dataset_train), len(dataset_val)

(1258, 223)

# 5. Setting up BERT Pretrained Model

In [27]:
# Importing Pre-Trained BERT Model

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(label_dict),
    output_attentions = False,      # For attending certain words above others
    output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# 6. Creating Data Loaders

In [25]:
# Creating Data Loaders for Train-Test

## Batch Size
batch_size = 32

## Data Loaders
dataloader_train = DataLoader(dataset_train, sampler = RandomSampler(dataset_train), batch_size = batch_size)
dataloader_val = DataLoader(dataset_val, sampler = RandomSampler(dataset_val), batch_size = batch_size)

# 7. Setting Up Optimizer and Scheduler

In [28]:
# Setting up Optimizer - Defining our LR and how it changes through each epoch [Stochastic Optimization Approach]
optimizer = AdamW(model.parameters(), lr = 1e-5, # Original BERT Paper Recommends between [2e-5, 5e-5]
                  eps = 1e-8
                  )

In [29]:
# Scheduler - Controls the Learning Rate
epochs = 10
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = len(dataloader_train) * epochs)

## num_training_steps - how many times we want our LR to change / no of steps

# 8. Defining our Performance Metrics

Note : Accuracy Metric Approach originally used in Accuracy Function in [this blog](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [30]:
# F1-Score Metric

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [31]:
# Accuracy per Class Metric

def accuracy_per_class(preds, labels):
    
    label_dict_inverse = {v : k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
      y_preds = preds_flat[labels_flat == label]
      y_true = labels_flat[labels_flat == label]
      print(f"Class : {label_dict_inverse[label]}")
      print(f"Accuracy : {len(y_preds[y_preds == label])} / {len(y_true)} \n")

# 9. Creating our Training Loop

Note : Approach Adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [32]:
# Setting up Random Seed for all the Libraries
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [33]:
# Sanity Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [34]:
# Evaluation Function

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().gpu().numpy()
        label_ids = inputs['labels'].gpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [35]:
# Training our BERT Model with Dataset

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()

    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, desc = 'Epoch {:1d}'.format(epoch), leave = False, disable = False)  # Check for Progress - How many Batches we've trained, how many to go

    for batch in progress_bar:                     # Batches for Back-Propagation

      model.zero_grad()                            # In transformers, we set the grad = 0 each time

      batch = tuple(b.to(device) for b in batch)

      inputs = {
          'input_ids'      :  batch[0],
          'attention_mask' :  batch[1],
          'labels'         :  batch[2]
      }

      outputs = model(**inputs)

      loss = outputs[0]
      loss_train_total += loss.item()
      loss.backward()

      # Clip our Gradients -> Takes out our Gradient and gives them a Norm Value
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      optimizer.step()
      scheduler.step()

      # Update the Progress of Training - Loss per Batch
      progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
    
    # Save the Model Every Epoch
    torch.save(model.state_dict(), f'Models/BERT_ft_epoch{epoch}.model')

    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write(f'Training Loss : {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation Loss : {val_loss}') 
    tqdm.write(f'F1 Score (Weighted) : {val_f1}')


Epoch 1
Training Loss : 1.2415860906243323
Validation Loss : 0.8774420022964478
F1 Score (Weighted) : 0.6656119824269878



Epoch 2
Training Loss : 0.7812037020921707
Validation Loss : 0.690807261637279
F1 Score (Weighted) : 0.7081066009897999



Epoch 3
Training Loss : 0.5738064363598824
Validation Loss : 0.5724027965750013
F1 Score (Weighted) : 0.770144838513869



Epoch 4
Training Loss : 0.44455542154610156
Validation Loss : 0.5529536945479256
F1 Score (Weighted) : 0.7861071829879738



Epoch 5
Training Loss : 0.3629385080188513
Validation Loss : 0.5419862014906747
F1 Score (Weighted) : 0.8107338248935051



Epoch 6
Training Loss : 0.30973140001296995
Validation Loss : 0.553049100296838
F1 Score (Weighted) : 0.818898576474579



Epoch 7
Training Loss : 0.2860778946429491
Validation Loss : 0.5542012474366597
F1 Score (Weighted) : 0.8300343627824205



Epoch 8
Training Loss : 0.25818761382251976
Validation Loss : 0.5607449401702199
F1 Score (Weighted) : 0.8365048628501542



Epoch 9
Training Loss : 0.23351746015250682
Validation Loss : 0.5651712162154061
F1 Score (Weighted) : 0.8438702543716767



Epoch 10
Training Loss : 0.22048840746283532
Validation Loss : 0.5641037097999028
F1 Score (Weighted) : 0.8438702543716767



# 10. Loading and Evaluating our Model

In [36]:
# Reloading our Model for Evaluation

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = len(label_dict), output_attentions = False, output_hidden_states = False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [37]:
# Move Model to Device
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [39]:
# Sanity Check for Mapping Location

if torch.cuda.is_available():
    map_location = lambda storage, loc: storage.cuda()
else:
    map_location = 'cpu'

print(f'Mapping Location : {map_location}')  # If Function Object --> Cuda else CPU

Mapping Location : <function <lambda> at 0x7f8d6c41a0d0>


In [40]:
# Checking our Available Models after Training
!ls 'Models'

BERT_ft_epoch10.model  BERT_ft_epoch6.model
BERT_ft_epoch1.model   BERT_ft_epoch7.model
BERT_ft_epoch2.model   BERT_ft_epoch8.model
BERT_ft_epoch3.model   BERT_ft_epoch9.model
BERT_ft_epoch4.model   finetuned_bert_epoch_1_gpu_trained.model
BERT_ft_epoch5.model


In [50]:
# Load in our Model
model.load_state_dict(torch.load('Models/finetuned_bert_epoch_1_gpu_trained.model', map_location = torch.device('cuda'))) 

# Note - Using Pre-Trained Model built on Another Notebook

<All keys matched successfully>

In [51]:
# Using Evaluate Function to take out Predictions
_, predictions, true_vals = evaluate(dataloader_val)

In [52]:
# Checking out Accuracy Per Class
accuracy_per_class(predictions, true_vals)

Class : happy
Accuracy : 163 / 171 

Class : not-relevant
Accuracy : 20 / 32 

Class : angry
Accuracy : 7 / 9 

Class : disgust
Accuracy : 0 / 1 

Class : sad
Accuracy : 4 / 5 

Class : surprise
Accuracy : 2 / 5 



# 11. Github Integration

In [53]:
# Initialising Empty Git Repo on Cloud
!git init

Initialized empty Git repository in /content/gdrive/My Drive/Data Science/NLP/BERT/.git/


In [54]:
# Adding email to the global config-file
!git config --global user.email rahulbordoloi24@gmail.com

# Adding username to the global config-file
!git config --global user.name rahulbordoloi

In [67]:
# Adding Remote Bert Repo
!git remote add bert https://rahulbordoloi:<personal-access-token>@github.com/rahulbordoloi/BERT-Transformer.git

In [68]:
# Fetching from Master
!git fetch bert master

From https://github.com/rahulbordoloi/BERT-Transformer
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> bert/master


In [69]:
# Pulling from Master
!git pull bert master

From https://github.com/rahulbordoloi/BERT-Transformer
 * branch            master     -> FETCH_HEAD
Already up to date.


In [72]:
# Adding and Commiting Files from Master

!git add 'Data'
!git commit -am "Dataset"
!git add 'Images'
!git commit -am "Images/PDF"

[master 35fcc2d] Dataset
 2 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 Images/BERT_diagrams.pdf
 create mode 100644 Images/BERT_diagrams.png
On branch master
Untracked files:
	Models/
	Project.ipynb

nothing added to commit but untracked files present


In [73]:
# Pushing to Git Repo
!git push -u bert master

Counting objects: 9, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (9/9), done.
Writing objects: 100% (9/9), 2.08 MiB | 6.16 MiB/s, done.
Total 9 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), done.
To https://github.com/rahulbordoloi/BERT-Transformer.git
   de40653..35fcc2d  master -> master
Branch 'master' set up to track remote branch 'master' from 'bert'.


# End